# 1. Convolution AutoEncoder(CAE) 
  - 정상 URL 모형 학습
  - 정상 URL의 재구축 오류 << 피싱 URL의 재구축 오류

# 2. Densely connected Convolutional Network(DanseNet)
- 재구축 오류의 Thresholding 값의 딥러닝 기반 학습

In [0]:
#Copyright 2019, Seokjun Bu, Softcomputing LAB all rights reserved.
%matplotlib inline
import keras
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model, Sequential
from keras.utils import np_utils, multi_gpu_model, plot_model
from keras.layers import Input, concatenate, Reshape, Flatten, Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Conv1D, MaxPool1D, UpSampling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE
from skimage.measure import compare_ssim as ssim
from tensorflow.keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D
from keras.layers import Input, Concatenate
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras import optimizers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = np.load('/content/drive/My Drive/test_colab/dataset_2_char_cat_45000_15000_140.npy')

In [4]:
# Train, Test Split
# Train : for CAE Learning + Detection Thresholding
# Test : only for validation
X, Y = data[:, 1:], data[:, 0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, shuffle=True, random_state=11)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(42000, 14000) (42000,)
(18000, 14000) (18000,)


In [0]:
Y_train_cat, Y_test_cat = to_categorical(Y_train), to_categorical(Y_test)

## 1. Bulid Convolution AutoEncoder

In [0]:
# cae_input = Input((X_train.shape[1], ))  # adapt this if using `channels_first` image data format
# H = Reshape((100, 140, 1))(cae_input)

# # Encoded
# H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
# H = MaxPooling2D((2,2))(H)
# H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
# H = MaxPooling2D((2,2))(H)

# # Decoded

# H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
# H = UpSampling2D((2, 2))(H)
# H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
# H = UpSampling2D((2, 2))(H)
# H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
# H = Conv2D(filters=1, kernel_size=(1, 1), padding='same', activation='relu')(H)

# cae_output = Flatten()(H)
# cae_model = Model(cae_input,cae_output)
# cae_model.summary()

# adam = Adam(lr = 1e-04)
# cae_model.compile(loss='categorical_crossentropy', 
#                   optimizer= adam, metrics=['accuracy'])
# cae_model.save('/content/drive/My Drive/test_colab/cae.h5')

In [6]:
cae_model = load_model('/content/drive/My Drive/test_colab/cae.h5')
cae_model.summary()














Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 14000)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 100, 140, 1)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 100, 140, 32)      160       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 50, 70, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 50, 70, 32)        4128      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 25, 35, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 25, 35, 32

In [7]:
# Lock Weight
for i, l in enumerate(cae_model.layers):
    l.trainable = False
    l.name = 'cae_'+str(i)
cae_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cae_0 (InputLayer)           (None, 14000)             0         
_________________________________________________________________
cae_1 (Reshape)              (None, 100, 140, 1)       0         
_________________________________________________________________
cae_2 (Conv2D)               (None, 100, 140, 32)      160       
_________________________________________________________________
cae_3 (MaxPooling2D)         (None, 50, 70, 32)        0         
_________________________________________________________________
cae_4 (Conv2D)               (None, 50, 70, 32)        4128      
_________________________________________________________________
cae_5 (MaxPooling2D)         (None, 25, 35, 32)        0         
_________________________________________________________________
cae_6 (Conv2D)               (None, 25, 35, 32)        4128

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


# 2. Bulid DenseNet

In [0]:
def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
  '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
        # Arguments
            x: input tensor 
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
  
  eps = 1.1e-5
  conv_name_base = 'conv' + str(stage) + '_' + str(branch)
  relu_name_base = 'relu' + str(stage) + '_' + str(branch)

  # 1x1 Convolution (Bottleneck layer)
  inter_channel = nb_filter * 4
  x = BatchNormalization(epsilon=eps, axis=3, name=conv_name_base+'_x1_bn')(x)
  x = Activation('relu', name=relu_name_base+'_x1')(x)
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)

  if dropout_rate:
    x = Dropout(dropout_rate)(x)

  # 3x3 Convolution
  x = BatchNormalization(epsilon=eps, axis=3, name=conv_name_base+'_x2_bn')(x)
  x = Activation('relu', name=relu_name_base+'_x2')(x)
  x = ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding')(x)
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)

  if dropout_rate:
    x = Dropout(dropout_rate)(x)

  return x

In [0]:
def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''
    # dense_block ; conv_block을 concat 한 것
    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = keras.layers.concatenate([concat_feat, x], name='concat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

In [0]:
def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
  ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
  eps = 1.1e-5

  conv_name_base = 'conv' + str(stage) + '_blk'
  relu_name_base = 'relu' + str(stage) + '_blk'
  pool_name_base = 'pool' + str(stage)

  x = BatchNormalization(epsilon=eps, axis=3, name=conv_name_base+'_bn')(x)
  x = Activation('relu', name=relu_name_base)(x)
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)

  if dropout_rate:
    x = Dropout(dropout_rate)(x)

  x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

  return x

In [11]:
cae_model.get_layer('cae_12').output

<tf.Tensor 'flatten_4/Reshape:0' shape=(?, ?) dtype=float32>

In [0]:
def DenseNet(nb_dense_block=4, growth_rate=48, nb_filter=96, reduction=0.0, dropout_rate=0.0,
             weight_decay=1e-4, classes=2):
  
  eps = 1.1e-5
  
  # transition layer에서 feature map의 수를 줄여주는 작업
  compression = 1.0 - reduction

  # From architecture for ImageNet (Table 1 in the paper)
  # nb_filter = 64
  nb_layers = [6, 12, 36, 24] # For DenseNet-121

  # Initial Convolution
  cae_output = cae_model.get_layer('cae_12').output
  x = Reshape((100, 140, 1))(cae_output)
  x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(x)
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
  x = BatchNormalization(epsilon=eps, axis=3, name='conv1_bn')(x)
  x = Activation('relu', name='pool1_zeropadding')(x)
  x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

  # Add dense blocks
  for block_idx in range(nb_dense_block -1):
    stage = block_idx+2
    x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
    
    # Add Transition Block
    x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
    nb_filter = int(nb_filter * compression)

  final_stage = stage + 1
  x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate,
                             weight_decay=weight_decay)
  
  x = BatchNormalization(epsilon=eps, axis=3, name='conv'+str(final_stage)+'_blk_bn')(x)
  x = Activation('relu', name='relu'+str(final_stage)+'_blk')(x)
  x = GlobalAveragePooling2D(name='pool'+str(final_stage))(x) #GAP는 파라미터 수를 줄여 오버피팅을 방지하는 기능
  x = Dense(classes, name='fc6')(x)
  x = Activation('softmax', name='prob')(x)

  model = Model(cae_model.input, x, name='dansenet')

  return model


In [13]:
densenet = DenseNet(nb_dense_block=2, growth_rate=4, nb_filter=8, reduction=0.5, dropout_rate=0.5, weight_decay=1e-4, classes=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (7, 7), name="conv1", strides=(2, 2), use_bias=False)`


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="conv2_1_x1", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), name="conv2_1_x2", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="conv2_2_x1", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), name="conv2_2_x2", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="conv2_3_x1", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` cal

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="conv2_blk", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="conv3_1_x1", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), name="conv3_1_x2", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="conv3_2_x1", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), name="conv3_2_x2", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call

In [17]:
adam = Adam(lr = 1e-04)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
densenet.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='auto', patience=10,restore_best_weights = True)

In [0]:
%%time
history_densenet = densenet.fit(X_train, Y_train_cat, validation_data=(X_test, Y_test_cat), 
                                                epochs=200, batch_size=32, verbose=2, callbacks = [es])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 42000 samples, validate on 18000 samples
Epoch 1/200
 - 171s - loss: 0.4478 - acc: 0.8055 - val_loss: 0.6449 - val_acc: 0.7526
Epoch 2/200
